In [14]:
from selenium import webdriver as wd
import numpy as np
import time
import shutil
import os
import pandas as pd

def stdt(t):
    w=t.split(':')
    for i in range(2):
        if int(w[i])<10:
            w[i]='0'+w[i]
        else:
            w[i]=w[i]
    return ''.join(w)
def slash(x):
    return x.replace('\\','/')

download_path=slash(r"C:\Users\Allen\Downloads")

def download_data(dates):
    cdir=slash(os.getcwd())
    for date in dates:
        #prepare for downloading
        if not os.path.exists(str(date)):
            os.makedirs(str(date))
            print(f'The folder {date} is created.')
        else:
            print(f'The folder {date} exists.')
            
        files=[]
        for i in range(24):
            for j in range(0,60,5):
                files.append(f'TDCS_M03A_{date}_'+stdt(f'{i}:{j}')+'00.csv')
        #download all files        
        if not os.path.exists(f'{date}/TDCS_M03A_{date}_000000.csv'):
            driver=wd.Chrome()
            url='https://tisvcloud.freeway.gov.tw/history/TDCS/M03A/'
            driver.maximize_window()
            driver.get(url)
            for i in range(2,1000):
                if driver.find_element('xpath',f'//*[@id="indexlist"]/tbody/tr[{i}]/td[2]/a').text.strip('/')==str(date):
                    driver.find_element('xpath',f'//*[@id="indexlist"]/tbody/tr[{i}]/td[2]/a').click()
                    break
            for i in range(25,1,-1):
                driver.find_element('xpath',f'//*[@id="indexlist"]/tbody/tr[{i}]/td[2]/a').click()
                for j in range(13,1,-1):
                    driver.find_element('xpath',f'//*[@id="indexlist"]/tbody/tr[{j}]/td[2]/a').click()
                driver.find_element('xpath','/html/body/div/div/section[2]/div/a[1]').click()
            driver.quit()
            try:
                for i in files:
                    shutil.move(f"{download_path}/{i}", f'{cdir}/{date}/')
            except:
                continue
def data_processing(dates,e):
    cdir=slash(os.getcwd())
    #車種：31小客車、32小貨車、41大客車、42大貨車、5聯結車
    pce_dict={'31':1,'32':1,'41':1.8,'42':1.8,'5':3}
    pce_list=list(pce_dict.values())
    pce_chinese_names=['小客車','小貨車','大客車','大貨車','聯結車']
    pce_chinese=dict(zip(pce_dict.keys(),pce_chinese_names))
    days_dict={}
    for date in dates:
        day_dicts=[]
        for t in range(24):
            hr_files=[]
            for j in range(0,60,5):
                hr_files.append(f'TDCS_M03A_{date}_'+stdt(f'{t}:{j}')+'00.csv')
            # hr_files.append(f'TDCS_M03A_{date}_'+stdt(f'{t}:{0}')+'00.csv')
            hr_dicts=[]
            for f in hr_files:
                #open 20240609/TDCS_M03A_20240609_000000.csv as two dim np array
                data=np.loadtxt(f'{date}/{f}',delimiter=',',dtype='str')
                data=data[:,[1,3,4]]
                #for data, create a dictionary with key as the first element of the row and value as tuple of the second and third elements of the row. if the key already exists, append the tuple to the list of the value of the key.
                d={}
                for i in data:
                    if i[0] in d:
                        d[i[0]].append((i[1],i[2]))
                    else:
                        d[i[0]]=[(i[1],i[2])]
                hr_dicts.append(d)
            #for all the element with the same keys in the dictionaries in hr_dicts, list the values of the elements and create a new dictionary with the same keys and the new values.
            hr_dict={}
            for i in hr_dicts:
                for j in i:
                    if j in hr_dict:
                        hr_dict[j].append(list(np.array(i[j])[:,1]))
                    else:
                        hr_dict[j]=[list(np.array(i[j])[:,1])]
            #for all the element with the same keys in the dictionaries in hr_dict, first convert all numbers to int, and then add up the values of the elements of the same column and create a new dictionary with the same keys and the new values.
            for i in hr_dict:
                for j in range(len(hr_dict[i])):
                    for k in range(len(hr_dict[i][j])):
                        hr_dict[i][j][k]=int(hr_dict[i][j][k])
                hr_dict[i]=list(np.sum(hr_dict[i],axis=0))
            day_dicts.append([t,hr_dict])
        #Consider each element in day_dicts a pair of time and dictionary. Create a new dictionary with the times as keys and the dictionaries as the values.
        day_dict={}
        for i in day_dicts:
            day_dict[i[0]]=i[1]
        #for each value of the values of day_dict, consider the list as traffic volume of each vehicle type. With the dictionary pce, multiply the traffic volume of each vehicle type and add up all the traffic volumes of each vehicle type to get pcu, and append the pcu to the list of pcu of the same key.
        for i in day_dict:
            for j in day_dict[i]:
                pcu=0
                for k in range(len(day_dict[i][j])):
                    pcu+=day_dict[i][j][k]*pce_list[k]
                #round the pcu to the nearest integer
                day_dict[i][j].append(round(pcu))
        
        #As we can see day_dict is a dictionary with keys as time and values as dictionary with keys as eTag ID and values as list of traffic volume of each vehicle type and pcu. 
        #For example, day_dict[6]['01F0750N'] is a list of traffic volume of each vehicle type and pcu during 5:05 to 6:00.
        #Now, we are going to create a dictionary with keys as eTag ID and values as list of traffic volume of each vehicle type and pcu during the day.
        day_dict2={}
        for i in e:
            day_dict2[i]=[]
            for j in day_dict:
                day_dict2[i].append(day_dict[j][i])
        #Now, day_dict2 is a dictionary with keys as eTag ID and values as list of traffic volume of each vehicle type and pcu during the day.
        days_dict[date]=day_dict2

    #As we can see days_dict is a dictionary with keys as date and values as dictionary with keys as eTag ID and values as list of traffic volume of each vehicle type and pcu. 
    #For example, days_dict[20240609]['01F0750N'] is a list of traffic volume of each vehicle type and pcu during the day of 2024/06/09.
    #Now, we are going to create a dictionary with keys as eTag ID and values as dictionary with keys as date and values as list of traffic volume of each vehicle type and pcu in the dictionary days_dict.
    days_dict2={}
    for i in e:
        days_dict2[i]={}
        for j in days_dict:
            days_dict2[i][j]=days_dict[j][i]
    #Now, days_dict2 is a dictionary with keys as eTag ID and values as dictionary with keys as date and values as list of traffic volume of each vehicle type and pcu in the dictionary days_dict.

    #for data in days_dict2, save xlsx files with the name of the key and the data as the content.
    #for the content, the name of the sheet is the key of the data.
    #the index is 'time', and the columns are the vehicle types and 'pcu'. attention, the vehicle types should be translated to Chinese based on the dictionary pce_chinese.
    #about the 'time', first create a list of '5:05-6:00', '6:05-7:00', ..., '22:05-23:00' for the name of the time.
    time_lst=[f'{i}:00-{i}:55' for i in range(24)]
    #all the files are saved in the folder named 'eTag分時資料_{date[0]}_{date[-1]}'
    folder_name=f'eTag分時資料_{dates[0]}-{dates[-1]}'
    if not os.path.exists(folder_name):
        os.makedirs(f'{cdir}/{folder_name}')
    for i in days_dict2:
        with pd.ExcelWriter(f'{cdir}/{folder_name}/{i}.xlsx') as writer:
            for j in days_dict2[i]:
                df=pd.DataFrame(days_dict2[i][j],columns=['小客車','小貨車','大客車','大貨車','聯結車','pcu'],index=time_lst)
                df.to_excel(writer,sheet_name=str(j),index_label='time')

In [15]:
e=['01F0256N', '01F0264S']
dates=[20210505]

In [16]:
download_data(dates)

The folder 20210505 is created.


In [20]:
data_processing(dates,e)